In [2]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:50000]").shuffle(seed=1234)
squad = squad.train_test_split(test_size=0.15)

squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 42500
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 7500
    })
})

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [5]:
from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)
data_collator = DefaultDataCollator()

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Map:   0%|          | 0/42500 [00:00<?, ? examples/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_args = TrainingArguments(
    output_dir="models",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

c:\Users\omarm\anaconda3\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
trainer.train()

  0%|          | 0/7971 [00:00<?, ?it/s]

{'loss': 2.9297, 'grad_norm': 16.884069442749023, 'learning_rate': 1.8745452264458662e-05, 'epoch': 0.19}
{'loss': 1.8735, 'grad_norm': 14.34164047241211, 'learning_rate': 1.7490904528917327e-05, 'epoch': 0.38}
{'loss': 1.4971, 'grad_norm': 13.684978485107422, 'learning_rate': 1.623635679337599e-05, 'epoch': 0.56}
{'loss': 1.379, 'grad_norm': 16.4865779876709, 'learning_rate': 1.4981809057834651e-05, 'epoch': 0.75}
{'loss': 1.3358, 'grad_norm': 14.200028419494629, 'learning_rate': 1.3727261322293316e-05, 'epoch': 0.94}


  0%|          | 0/469 [00:00<?, ?it/s]

{'eval_loss': 1.1639306545257568, 'eval_runtime': 35.6569, 'eval_samples_per_second': 210.338, 'eval_steps_per_second': 13.153, 'epoch': 1.0}
{'loss': 1.1227, 'grad_norm': 20.447202682495117, 'learning_rate': 1.2472713586751976e-05, 'epoch': 1.13}
{'loss': 1.0262, 'grad_norm': 19.58578872680664, 'learning_rate': 1.1218165851210639e-05, 'epoch': 1.32}
{'loss': 1.0358, 'grad_norm': 14.324488639831543, 'learning_rate': 9.963618115669303e-06, 'epoch': 1.51}
{'loss': 1.0013, 'grad_norm': 20.745166778564453, 'learning_rate': 8.709070380127964e-06, 'epoch': 1.69}
{'loss': 0.9899, 'grad_norm': 14.19008731842041, 'learning_rate': 7.454522644586627e-06, 'epoch': 1.88}


  0%|          | 0/469 [00:00<?, ?it/s]

{'eval_loss': 1.0875186920166016, 'eval_runtime': 36.6458, 'eval_samples_per_second': 204.662, 'eval_steps_per_second': 12.798, 'epoch': 2.0}
{'loss': 0.9473, 'grad_norm': 10.1917142868042, 'learning_rate': 6.19997490904529e-06, 'epoch': 2.07}
{'loss': 0.7932, 'grad_norm': 9.989960670471191, 'learning_rate': 4.945427173503952e-06, 'epoch': 2.26}
{'loss': 0.8037, 'grad_norm': 8.205730438232422, 'learning_rate': 3.690879437962615e-06, 'epoch': 2.45}
{'loss': 0.7936, 'grad_norm': 12.738724708557129, 'learning_rate': 2.4363317024212772e-06, 'epoch': 2.63}
{'loss': 0.8036, 'grad_norm': 9.948616027832031, 'learning_rate': 1.1817839668799399e-06, 'epoch': 2.82}


  0%|          | 0/469 [00:00<?, ?it/s]

{'eval_loss': 1.1168338060379028, 'eval_runtime': 36.5198, 'eval_samples_per_second': 205.368, 'eval_steps_per_second': 12.842, 'epoch': 3.0}
{'train_runtime': 1974.6452, 'train_samples_per_second': 64.569, 'train_steps_per_second': 4.037, 'train_loss': 1.1971493013798555, 'epoch': 3.0}


TrainOutput(global_step=7971, training_loss=1.1971493013798555, metrics={'train_runtime': 1974.6452, 'train_samples_per_second': 64.569, 'train_steps_per_second': 4.037, 'total_flos': 1.249370269056e+16, 'train_loss': 1.1971493013798555, 'epoch': 3.0})